In [14]:

from umap_script import loadHSI, show_clusterable_embedding, visualize_umap, compare_umap    
from sklearn.neighbors import NearestNeighbors
import math

import numpy as np
import random
import matplotlib.pyplot as plt
import umap

In [11]:
salinasA_path = 'data/SalinasA_corrected.mat'
salinasA_gt_path = 'data/SalinasA_gt.mat'

X, M, N, D, HSI, GT, Y, n, K = loadHSI(salinasA_path, salinasA_gt_path, 'salinasA_corrected', 'salinasA_gt')

83


In [ ]:
def estAdditiveNoise(r, verbose = 'on'):
    verbose = 0
    small = 1e-6
    verbose = 0
    L, N = r.shape
    w = np.zeros((L, N))
    if verbose == 'on':
        print('computing the sample correlation matrix and its inverse')
    RR = np.dot(r, r.T) 
    if np.linalg.cond(RR + small * np.eye(L)) < 1e-8:
    # near singular, so we double perturbation size.
        small = 5e-5
    RRi = np.linalg.inv(RR + small * np.eye(L))
    if verbose:
        print("computing band")
    for i in range(1, L+1):
        if verbose:
            print('\b\b\b%3d',i)
        XX = RRi - (np.outer(RRi[:, i], RRi[i, :])) / RRi[i, i]
        RRa = RR[:, i].copy()
        RRa[i] = 0
        
        beta = np.dot(XX, RRa)
        beta[i] = 0
        # need to compute np.dot(beta', r) i.e., conjugate of beta - w(i,:) = r(i,:) - beta'*r;
        w[i, :] = r[i, :] - np.dot(np.conj(beta),r)
    if verbose == 'on':
        print('\ncomputing noise correlation matrix')
    Rw = np.diag(np.diag(np.dot(w, w.T) / N))
    return w, Rw

In [ ]:
def hysime(*args):
#     function [varargout]=hysime(varargin)
    """HySime: Hyperspectral signal subspace estimation

    [kf,Ek]=hysime(y,w,Rw,verbose);

    Input:
        y  hyperspectral data set (each column is a pixel)
            with (L x N), where L is the number of bands
            and N the number of pixels
        w  (L x N) matrix with the noise in each pixel
        Rw noise correlation matrix (L x L)
        verbose [optional] (on)/off
    Output
        kf signal subspace dimension
        Ek matrix which columns are the eigenvectors that span 
            the signal subspace

    Copyright: Jose Nascimento (zen@isel.pt)
                & 
                Jose Bioucas-Dias (bioucas@lx.it.pt)

    For any comments contact the authors"""
    if len(args) < 1 or len(args) > 3:
        raise ValueError("Invalid number of input arguments. Expected between 1 and 3.")

    if len(args) > 2:
        raise ValueError("Too many output parameters")

    y = args[0]
    if not isinstance(y, np.ndarray):
        raise ValueError("The data set must be an L x N matrix")
    
    noise_type = 'additive'
    verbose = 1
    verb = 'on'
    
    for i in range(1, len(args)):
        arg = args[i].lower()
        if arg == 'additive':
            noise_type = 'additive'
        elif arg == 'poisson':
            noise_type = 'poisson'
        elif arg == 'on':
            verbose = 1
            verb = 'on'
        elif arg == 'off':
            verbose = 0
            verb = 'off'
        else:
            raise ValueError(f"Parameter [{i+1}] is unknown")
    verbose = 0

    
    L,N = y.shape
    if L<2:
        raise ValueError('Too few bands to estimate the noise.')

    if verbose:
        print('Noise estimates:')

    if (noise_type == 'poisson'):
        # sqy = np.sqrt(y*(y>0));     
        sqy = np.sqrt(np.multiply(y,(y>0)))    
        u,Ru = estAdditiveNoise(sqy,verb)
        x = np.square((sqy - u))           
        # w = sqrt(x).*u*2;
        w = np.matmul((np.multiply(np.sqrt(x), u)),2)
        # Rw = w*w'/N; 
        Rw = np.dot(w, np.conj(w)) / N
    else:
        w,Rw = estAdditiveNoise(y,verb)
    Ln,Nn = w.shape
    d1,d2 = Rw.shape

    if ~(Ln==L) or ~(Nn==N):
    # % n is an empty matrix or with different size:
        raise ValueError('empty noise matrix or its size does not agree with size of y\n')
    
    if ~(d1==d2) or ~(d1==L):
        print('Bad noise correlation matrix\n')
        # Rw = w*w'/N;     
        Rw = np.dot(w, np.conj(w)) / N
    
    x = y - w
    if verbose:
        print("Computing the correlation matrices")
    L, N = y.shape
    # Ry is a sample correlation matrix
    # Rx is a signal correlation matrix estimates
    Ry = np.dot(y, np.conj(y)) / N 
    Rx = np.dot(x, np.conj(x)) / N
    if verbose:
        print("Computing the eigenvectors of the signal correlation matrix")
    E, D = np.linalg.svd(Rx)
    dx = np.diag
    
    if verbose:
        print("Estimating the number of endmembers")
    Rw = Rw + np.sum(np.diag(Rx)) / L / 10**5 * np.eye(L)
    
    Py = np.diag(np.dot(np.dot((np.conj(E)),Ry),E))
    Rw = np.diag(np.dot(np.dot((np.conj(E)),Rw),E))
    cost_F = -Py + np.matmul(2,Rw)
    kf = np.sum(cost_F < 0)
    ind_asc = np.argsort(cost_F)
    Ek = E[:, ind_asc[:kf]]
    if verbose == 'on':
        print(f'The signal subspace dimension is: k = {kf}')

    # varargout(1) = {kf};
    # if nargout == 2, varargout(2) = {Ek};end
    # return
    outputs = [kf]
    if nargout == 2:
        outputs.append(Ek)
    
    return outputs

In [ ]:
def load_hyperparameters(HSI, HSIName, AlgName):
    # Define necessary variables and functions
    def hysime(X):
        # Define your hysime function here
        pass

    # Assume X, M, N, and AlgName are defined
    if AlgName == 'D-VIC':
        # Perform k-nearest neighbors search
        nbrs = NearestNeighbors(n_neighbors=1000, algorithm='auto').fit(X)
        Dist_NN, _ = nbrs.kneighbors(X)

        # Determine NN, pct, and K based on HSIName
        if HSIName == 'Synthetic Data':
            NN = 320
            pct = 10.5
            K = 3
        elif HSIName == 'Salinas A':
            NN = 30
            pct = 96.3157894736842
            K = 6
        elif HSIName == 'Jasper Ridge':
            NN = 20
            pct = 92.75862068965517
            K = 4
        elif HSIName == 'Indian Pines':
            NN = 40
            pct = 76.0526315789474
            K = 16

        # Set default parameters
        Hyperparameters = {}
        Hyperparameters['SpatialParams'] = {'ImageSize': (M, N)}
        Hyperparameters['NEigs'] = 10
        Hyperparameters['NumDtNeighbors'] = 200
        Hyperparameters['Beta'] = 2
        Hyperparameters['Tau'] = 10**(-5)
        Hyperparameters['Tolerance'] = 1e-8
        Hyperparameters['K_Known'] = K
        Hyperparameters['EndmemberParams'] = {
            'Algorithm': 'ManyAVMAX',
            'NumReplicates': 100,
            'K': hysime(X.T)  # Assuming hysime is defined elsewhere
        }
        Hyperparameters['DiffusionNN'] = NN
        Hyperparameters['DensityNN'] = NN  # Must be ≤ 1000
        Hyperparameters['Sigma0'] = np.percentile(Dist_NN[Dist_NN > 0], pct, interpolation='nearest')

        print(Hyperparameters)